In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd

from surprise import BaselineOnly, Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate, train_test_split

In [3]:
df = pd.read_parquet('../../data/foursquare/interactions.parquet')

In [4]:
df

,userId,venueCategoryId,timestamp,relevant
0,470,4bf58dd8d48988d127951735,1.333462e+09,1
1,979,4bf58dd8d48988d1df941735,1.333462e+09,1
2,69,4bf58dd8d48988d103941735,1.333462e+09,1
3,395,4bf58dd8d48988d104941735,1.333462e+09,1
4,87,4bf58dd8d48988d1cb941735,1.333462e+09,1
...,...,...,...,...
227423,688,4bf58dd8d48988d1e7931735,1.360964e+09,1
227424,560,4bf58dd8d48988d16c941735,1.360964e+09,1
227425,945,4bf58dd8d48988d103941735,1.360964e+09,1
227426,671,4bf58dd8d48988d11d941735,1.360964e+09,1


In [5]:
df.describe()

,userId,timestamp,relevant
count,227428.000000,2.274280e+05,227428.0
mean,535.304817,1.343574e+09,1.0
std,305.398774,8.450872e+06,0.0
min,1.000000,1.333462e+09,1.0
25%,287.000000,1.336547e+09,1.0
50%,523.000000,1.340033e+09,1.0
75%,793.000000,1.351790e+09,1.0
max,1083.000000,1.360964e+09,1.0


In [33]:
reader = Reader(rating_scale=(1, 1))
data = Dataset.load_from_df(df[['userId', 'venueCategoryId', 'relevant']], reader)

In [7]:
train, test = train_test_split(data, )

In [30]:
df[['userId', 'venueCategoryId', 'relevant']]

,userId,venueCategoryId,relevant
0,470,4bf58dd8d48988d127951735,1
1,979,4bf58dd8d48988d1df941735,1
2,69,4bf58dd8d48988d103941735,1
3,395,4bf58dd8d48988d104941735,1
4,87,4bf58dd8d48988d1cb941735,1
...,...,...,...
227423,688,4bf58dd8d48988d1e7931735,1
227424,560,4bf58dd8d48988d16c941735,1
227425,945,4bf58dd8d48988d103941735,1
227426,671,4bf58dd8d48988d11d941735,1


In [28]:
train

In [35]:
data

In [12]:
sim_options = {"name": "cosine", "user_based": True}

In [13]:
algo = KNNBasic(sim_options=sim_options)

In [36]:
algo.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
predictions = algo.test(test)

In [16]:
predictions

[Prediction(uid=786, iid='4bf58dd8d48988d164941735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=705, iid='4bf58dd8d48988d103941735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=976, iid='4bf58dd8d48988d1fd931735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=39, iid='4bf58dd8d48988d114951735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=593, iid='4bf58dd8d48988d1fe931735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=680, iid='4bf58dd8d48988d175941735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=863, iid='4bf58dd8d48988d125941735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=579, iid='4bf58dd8d48988d1fd931735', r_ui=1.0, est=1, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=202, iid=

In [17]:
import pandas as pd

In [19]:
q = pd.DataFrame(predictions)

In [24]:
q

,uid,iid,r_ui,est,details
0,786,4bf58dd8d48988d164941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
1,705,4bf58dd8d48988d103941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
2,976,4bf58dd8d48988d1fd931735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
3,39,4bf58dd8d48988d114951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
4,593,4bf58dd8d48988d1fe931735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
...,...,...,...,...,...
45481,645,4bf58dd8d48988d129951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
45482,175,4bf58dd8d48988d118951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
45483,943,4bf58dd8d48988d103941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
45484,422,4bf58dd8d48988d1a0941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"


In [25]:
k=100
print(q.groupby('uid').apply(lambda dfg: dfg.nlargest(k,'r_ui'))
           .reset_index(drop=True))
#q.groupby('uid').head(2).reset_index(drop=True)

        uid                       iid  r_ui  est  \
0         1  4bf58dd8d48988d103951735   1.0    1   
1         1  4bf58dd8d48988d16c941735   1.0    1   
2         1  4bf58dd8d48988d10d951735   1.0    1   
3         1  4bf58dd8d48988d123941735   1.0    1   
4         1  4bf58dd8d48988d157941735   1.0    1   
...     ...                       ...   ...  ...   
41923  1083  4bf58dd8d48988d129951735   1.0    1   
41924  1083  4bf58dd8d48988d11b941735   1.0    1   
41925  1083  4bf58dd8d48988d124941735   1.0    1   
41926  1083  4bf58dd8d48988d147941735   1.0    1   
41927  1083  4bf58dd8d48988d124941735   1.0    1   

                                         details  
0      {'actual_k': 40, 'was_impossible': False}  
1      {'actual_k': 40, 'was_impossible': False}  
2      {'actual_k': 40, 'was_impossible': False}  
3      {'actual_k': 40, 'was_impossible': False}  
4      {'actual_k': 40, 'was_impossible': False}  
...                                          ...  
41923  {'actual_k'

In [27]:
q.groupby('uid').apply(lambda dfg: dfg.nlargest(k,'r_ui')).reset_index(drop=True)

,uid,iid,r_ui,est,details
0,1,4bf58dd8d48988d103951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
1,1,4bf58dd8d48988d16c941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
2,1,4bf58dd8d48988d10d951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
3,1,4bf58dd8d48988d123941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
4,1,4bf58dd8d48988d157941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
...,...,...,...,...,...
41923,1083,4bf58dd8d48988d129951735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
41924,1083,4bf58dd8d48988d11b941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
41925,1083,4bf58dd8d48988d124941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
41926,1083,4bf58dd8d48988d147941735,1.0,1,"{'actual_k': 40, 'was_impossible': False}"
